# Dependences

In [3]:
include("../../src/struct_data.jl")

LoadError: LoadError: syntax: missing comma or ) in argument list
in expression starting at /home/nicomosty/Documentos/Maestria-GitHub/CellAggregate.jl/src/struct_data.jl:314

# Force Functions

## Cubic

In [ ]:
@make_struct_func Cubic
dump(Cubic)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: Cubic{T} <: ForceType
    μ₁::T
    rₘᵢₙ::T
    rₘₐₓ::T


In [ ]:
# Parameters
μ₁   = 1.0
rₘₐₓ = 2.0
rₘᵢₙ = 3.0

3.0

In [ ]:
Cubic(μ₁,rₘₐₓ,rₘᵢₙ)

Cubic{Float64}(1.0, 2.0, 3.0)

In [ ]:
# Show function on specific r value
r = 1
force_func(Cubic(μ₁,rₘₐₓ,rₘᵢₙ),r)

4.0

## LennardJones

In [ ]:
@make_struct_func LennardJones
dump(LennardJones)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: LennardJones{T} <: ForceType
    μ₁::T
    rₘᵢₙ::T
    rₘₐₓ::T


In [ ]:
# Parameters
μ₁   = 1.0
rₘₐₓ = 2.0
rₘᵢₙ = 3.0

3.0

In [ ]:
LennardJones(μ₁,rₘₐₓ,rₘᵢₙ)

LennardJones{Float64}(1.0, 2.0, 3.0)

In [ ]:
# Show function on specific r value
r = 1.9
force_func(LennardJones(μ₁,rₘₐₓ,rₘᵢₙ),r)

1.9609746568876094

# Aggregate

## Inizializate (Model Parameter)

In [ ]:
Model = ModelPar(
    TimePar(
        t_f      = 10000.0,
        dt       = 0.5
    ),
    NeighborPar(n_knn = 50),
    GeometryPar(
        r_agg    = 13.0,
        position = [0 0 0;
                    1 0 0],
        outer_ratio = 0.7
    ),
    SimulationPar(
        n_text     = 100,
        path_input = "../../data/init/Sphere",
        path_ouput = "AAA",
        name_cell  = "Queso"
    )
)

ModelPar(TimePar(10000.0, 0.5), NeighborPar(50), GeometryPar(13.0, [0 0 0; 1 0 0], 0.7), SimulationPar(100, "../../data/init/Sphere", "AAA", "Queso"))

## One Agg

In [ ]:
AGG = Nothing
@time AGG = OneAgg(Model, Cubic(1,2,3), ContractilePar(1))
show_agg(AGG)

  0.207682 seconds (279.06 k allocations: 16.261 MiB, 6.45% gc time)
===== Agg Position Matrix ======


1620×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.5  -4.04  -12.25
  1.5  -4.04  -12.25
 -3.5  -2.31  -12.25
 -1.5  -2.31  -12.25
  0.5  -2.31  -12.25
  2.5  -2.31  -12.25
 -2.5  -0.58  -12.25
 -0.5  -0.58  -12.25
  1.5  -0.58  -12.25
  3.5  -0.58  -12.25
 -3.5   1.15  -12.25
 -1.5   1.15  -12.25
  0.5   1.15  -12.25
  ⋮           
  0.5  -1.15   12.25
  2.5  -1.15   12.25
 -2.5   0.58   12.25
 -0.5   0.58   12.25
  1.5   0.58   12.25
  3.5   0.58   12.25
 -3.5   2.31   12.25
 -1.5   2.31   12.25
  0.5   2.31   12.25
  2.5   2.31   12.25
 -0.5   4.04   12.25
  1.5   4.04   12.25

========== Agg Radius ==========
r_agg      = 12.983333333333334
====== Position of Aggregate =====


1×3 Matrix{Int64}:
 0  0  0

===== Position Matrix Size =====
X          = (1620, 3)
dX         = (1620, 3)
===== Neighbors Matrix Size ====
idx        = (1620, 1620)
idx_red    = (21, 1620)
idx_sum    = (1, 1620)
idx_cont   = (1, 1620)
======= Force Matrix Size =======
F          = (1620, 3)
======= Index Aggregate =========


1×1620 adjoint(::Matrix{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

===== Outer vs Inner Cells =====


1×1620 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.6580246913580247


## Two Aggs

In [ ]:
# @time AGG = MoreAgg(AGG,Model, Cubic(1,2,3), ContractilePar(1))
# show_agg(AGG)

## Fusion Agg

In [ ]:
@time AGG = FusionAgg(AGG,Model, Cubic(1,2,3), ContractilePar(1))
show_agg(AGG)

  0.139296 seconds (202.67 k allocations: 12.429 MiB, 99.46% compilation time)
===== Agg Position Matrix ======


3240×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -13.4833   -4.04  -12.25
 -11.4833   -4.04  -12.25
 -16.4833   -2.31  -12.25
 -14.4833   -2.31  -12.25
 -12.4833   -2.31  -12.25
 -10.4833   -2.31  -12.25
 -15.4833   -0.58  -12.25
 -13.4833   -0.58  -12.25
 -11.4833   -0.58  -12.25
  -9.48333  -0.58  -12.25
 -16.4833    1.15  -12.25
 -14.4833    1.15  -12.25
 -12.4833    1.15  -12.25
   ⋮               
  13.4833   -1.15   12.25
  15.4833   -1.15   12.25
  10.4833    0.58   12.25
  12.4833    0.58   12.25
  14.4833    0.58   12.25
  16.4833    0.58   12.25
   9.48333   2.31   12.25
  11.4833    2.31   12.25
  13.4833    2.31   12.25
  15.4833    2.31   12.25
  12.4833    4.04   12.25
  14.4833    4.04   12.25

========== Agg Radius ==========
r_agg      = 12.983333587646484
====== Position of Aggregate =====


2×3 Matrix{Float64}:
 -12.9833  0.0  0.0
  12.9833  0.0  0.0

===== Position Matrix Size =====
X          = (3240, 3)
dX         = (3240, 3)
===== Neighbors Matrix Size ====
idx        = (3240, 3240)
idx_red    = (21, 3240)
idx_sum    = (1, 3240)
idx_cont   = (1, 3240)
======= Force Matrix Size =======
F          = (3240, 3)
======= Index Aggregate =========


1×3240 adjoint(::Matrix{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2

===== Outer vs Inner Cells =====


1×3240 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.6580246913580247
